# Breast Cancer Classification

# Import packages

In [1]:
#!pip install scikit-learn opencv-python 

In [2]:
#pip install torch==2.0+cu117 --user -f https://download.pytorch.org/whl/cu117/torch_stable.html

In [3]:
#!pip install torchvision==0.15.0

In [4]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, ConcatDataset
import glob
#import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score
import random
import cv2
import sys
from torchvision import models,transforms,datasets
import time
import os
import torch.nn.functional as F
from torchvision.datasets.folder import default_loader
import torchvision
from sklearn .metrics import roc_auc_score,accuracy_score,precision_score,recall_score,f1_score
from PIL import Image
from torch.autograd import Variable
import torch.optim.lr_scheduler as lr_scheduler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [5]:
print(torch.cuda.is_available()) # this should return true
torch.cuda.get_device_name() # this should return your graphics card name. Ex) 'NVIDIA RTX A4000'

True


'NVIDIA RTX A4000'

In [6]:
# # importing the zipfile module 
"""
from zipfile import ZipFile 

 # loading the temp.zip and creating a zip object 
with ZipFile("C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\cbis_ddsm.zip", 'r') as zObject: 
    zObject.extractall() 
"""

'\nfrom zipfile import ZipFile \n\n # loading the temp.zip and creating a zip object \nwith ZipFile("C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\cbis_ddsm.zip", \'r\') as zObject: \n    zObject.extractall() \n'

In [7]:
import pandas as pd
dicom_data = pd.read_csv("C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\csv\\dicom_info.csv")

full_mammogram_images = dicom_data[dicom_data.SeriesDescription == 'full mammogram images'].image_path
full_mammogram_images.head()

1     CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.248386...
2     CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.267213...
11    CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.210396...
12    CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.749566...
15    CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.987658...
Name: image_path, dtype: object

In [49]:
# load the mass dataset
mass_train = pd.read_csv('C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\csv\\mass_case_description_train_set.csv')
mass_test = pd.read_csv('C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\csv\\mass_case_description_test_set.csv')

#mass_train.head()
mass_train.iloc[0][11].split("/")[2]

# fix image paths
def fix_image_path(data):
 #   correct dicom paths to correct image paths
  for index, img in enumerate(data.values):
    img_name = img[11].split("/")[2]
    new_path = "C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\jpeg\\" + img_name
    data.iloc[index,11] = new_path
      
# apply to datasets
fix_image_path(mass_train)
fix_image_path(mass_test)


C:\Users\ae423\AppData\Local\Temp\ipykernel_23504\2910366824.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  mass_train.iloc[0][11].split("/")[2]


In [48]:

calc_train = pd.read_csv('C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\csv\\calc_case_description_train_set.csv')
calc_test = pd.read_csv('C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\csv\\calc_case_description_test_set.csv')

fix_image_path(calc_train)
fix_image_path(calc_test)


In [56]:
temp_calc_train = calc_train[["image file path","pathology"]]
calc_normal = []
calc_malignant = []
for row,col in temp_calc_train.iterrows():
    if col["pathology"] == "MALIGNANT":
        calc_malignant.append(col["image file path"])
    else:
        calc_normal.append(col["image file path"])

print(len(calc_normal))
print(len(calc_malignant))
print(len(temp_calc_train))


1002
544
1546


In [57]:
new_df = mass_train[["image file path","pathology"]]

normal_path = []
malignant_path = []
for row,col in new_df.iterrows():
    if col["pathology"] == "MALIGNANT":
        # put this file to the malignant folder
        malignant_path.append(col["image file path"])
    else:
        normal_path.append(col["image file path"])

print(len(normal_path))
print(len(malignant_path))
print(len(new_df))


681
637
1318


In [46]:
path = "C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\"
print(len(os.listdir(os.path.join(path,"cbis-test\\malignant"))))
print(len(os.listdir(os.path.join(path,"cbis-test\\normal"))))
print(len(os.listdir(os.path.join(path,"cbis-train\\malignant"))))
print(len(os.listdir(os.path.join(path,"cbis-train\\normal"))))
print(len(os.listdir(os.path.join(path,"cbis-val\\malignant"))))
print(len(os.listdir(os.path.join(path,"cbis-val\\normal"))))


104
128
295
296
86
125


In [66]:
"""
from sklearn.model_selection import train_test_split


temp,test = train_test_split(last_df, test_size=0.1,shuffle= True)
train,val = train_test_split(temp, test_size=0.1,shuffle= True)

print(len(train))
print(len(val))
print(len(test))
"""
last_df = pd.concat([temp_calc_train,new_df])
calc_test = calc_test[["image file path","pathology"]]
mass_test = mass_test[["image file path","pathology"]]
new_test = pd.concat([calc_test,mass_test])

train = last_df[:2160]
val = last_df[2160:]
test = new_test


,image file path,pathology
0,C:\Users\ae423\OneDrive - University of Sussex...,BENIGN
1,C:\Users\ae423\OneDrive - University of Sussex...,BENIGN
2,C:\Users\ae423\OneDrive - University of Sussex...,BENIGN_WITHOUT_CALLBACK
3,C:\Users\ae423\OneDrive - University of Sussex...,BENIGN_WITHOUT_CALLBACK
4,C:\Users\ae423\OneDrive - University of Sussex...,BENIGN_WITHOUT_CALLBACK
...,...,...
373,C:\Users\ae423\OneDrive - University of Sussex...,BENIGN_WITHOUT_CALLBACK
374,C:\Users\ae423\OneDrive - University of Sussex...,MALIGNANT
375,C:\Users\ae423\OneDrive - University of Sussex...,MALIGNANT
376,C:\Users\ae423\OneDrive - University of Sussex...,MALIGNANT


In [13]:
"""
import shutil

def write_into_train_folder():
    for row,col in train.iterrows():
        for img_name in os.listdir(col["image file path"]):
            img_path = col["image file path"] + "\\" + img_name
            if col["pathology"] == "MALIGNANT":
                # put this file to the malignant folder
                shutil.copy(img_path,"C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\cbis-train\\malignant")
            else:
                shutil.copy(img_path,"C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\cbis-train\\normal")
                # put it into normal folder


def write_into_val_folder():
    for row,col in val.iterrows():
        for img_name in os.listdir(col["image file path"]):
            img_path = col["image file path"] + "\\" + img_name
            if col["pathology"] == "MALIGNANT":
                # put this file to the malignant folder
                shutil.copy(img_path,"C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\cbis-val\\malignant")
            else:
                shutil.copy(img_path,"C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\cbis-val\\normal")
                # put it into normal folder


def write_into_test_folder():
    for row,col in test.iterrows():
        for img_name in os.listdir(col["image file path"]):
            img_path = col["image file path"] + "\\" + img_name
            if col["pathology"] == "MALIGNANT":
                # put this file to the malignant folder
                shutil.copy(img_path,"C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\cbis-test\\malignant")
            else:
                shutil.copy(img_path,"C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\cbis-test\\normal")
                # put it into normal folder

write_into_train_folder()
write_into_val_folder()
write_into_test_folder()
"""


'\nimport shutil\n\ndef write_into_train_folder():\n    for row,col in train.iterrows():\n        for img_name in os.listdir(col["image file path"]):\n            img_path = col["image file path"] + "\\" + img_name\n            if col["pathology"] == "MALIGNANT":\n                # put this file to the malignant folder\n                shutil.copy(img_path,"C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\cbis-train\\malignant")\n            else:\n                shutil.copy(img_path,"C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\cbis-train\\normal")\n                # put it into normal folder\n\n\ndef write_into_val_folder():\n    for row,col in val.iterrows():\n        for img_name in os.listdir(col["image file path"]):\n            img_path = col["image file path"] + "\\" + img_name\n            if col["pathology"] == "MALIGNANT":\n                # put this file to the malignant folder\n                shu

In [14]:
"""
import os 
#os.makedirs("cbis-train/normal")
os.makedirs("cbis-train/malignant")
os.makedirs("cbis-val/normal")
os.makedirs("cbis-val/malignant")
os.makedirs("cbis-test/normal")
os.makedirs("cbis-test/malignant")

def write_inside_folder(df):
    for folder in df:
        for file in os.listdir(folder):
            cv2.imwrite(folder,file)

#write_inside_folder(train)
#write_inside_folder(val)
#write_inside_folder(test)
"""

'\nimport os \n#os.makedirs("cbis-train/normal")\nos.makedirs("cbis-train/malignant")\nos.makedirs("cbis-val/normal")\nos.makedirs("cbis-val/malignant")\nos.makedirs("cbis-test/normal")\nos.makedirs("cbis-test/malignant")\n\ndef write_inside_folder(df):\n    for folder in df:\n        for file in os.listdir(folder):\n            cv2.imwrite(folder,file)\n\n#write_inside_folder(train)\n#write_inside_folder(val)\n#write_inside_folder(test)\n'

In [15]:
import wandb
run = wandb.init(
    # Set the project where this run will be logged
    project="Resnet50 Output",
    # Track hyperparameters and run metadata
)
wandb.login()


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: alphanhilal (alphanhilal-university-of-sussex). Use `wandb login --relogin` to force relogin


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [16]:
class CLAHE():
    
    def __init__(self, clipLimit=2.0, tileGridSize=(16,16)):
        super().__init__
        self.clipLimit = clipLimit
        self.tileGridSize = tileGridSize

    def __call__(self,sample):
      img = np.array(sample)
      img = np.moveaxis(img,0,2)
      imgray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
      imgray = imgray.astype(np.uint16)
      clahe = cv2.createCLAHE(clipLimit=self.clipLimit, tileGridSize=self.tileGridSize)
      new_img = clahe.apply(imgray)
      new_img = cv2.cvtColor(new_img, cv2.COLOR_GRAY2BGR)
      new_img = cv2.cvtColor(new_img, cv2.COLOR_BGR2RGB)
      new_img = new_img.astype(np.float32)
      new_img = np.moveaxis(new_img,2,0)
      tensor = torch.tensor(new_img)
      #print(img.shape)
      return tensor


class GaussianNoise():

    def __init__(self,mean=0,stddev=0.01):
        super().__init__
        self.mean = mean
        self.stddev = stddev

    def __call__(self,sample):
        sample = sample.cpu()
        input_array = sample.data.numpy()

        noise = np.random.normal(loc=self.mean, scale=self.stddev, size=np.shape(input_array))

        out = np.add(input_array, noise)

        output_tensor = torch.from_numpy(out)
        out_tensor = Variable(output_tensor)
        out = out_tensor.cuda()
        out = out.float()
        return out

In [17]:
import copy

class EarlyStopping:
    def __init__(self, patience=5, min_delta=0, restore_best_weights=True):
        self.patience = patience
        self.min_delta = min_delta
        self.restore_best_weights = restore_best_weights
        self.best_model = None
        self.best_loss = None
        self.counter = 0
        self.status = ""

    def __call__(self, model, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.best_model = copy.deepcopy(model.state_dict())
        elif self.best_loss - val_loss >= self.min_delta:
            self.best_model = copy.deepcopy(model.state_dict())
            self.best_loss = val_loss
            self.counter = 0
            self.status = f"Improvement found, counter reset to {self.counter}"
        else:
            self.counter += 1
            self.status = f"No improvement in the last {self.counter} epochs"
            if self.counter >= self.patience:
                self.status = f"Early stopping triggered after {self.counter} epochs."
                if self.restore_best_weights:
                    model.load_state_dict(self.best_model)
                return True
        return False

In [18]:
resnet_predictions = [] # The outputs of predictions will be stored in this array
numerical_predictions = [] # Numerical predictions will be stored in this array

In [34]:
def build_dataset(batch_size):
   
    transforms_train = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(p=0.5), # data augmentation
    transforms.RandomRotation(degrees=(0,180)),
    transforms.RandomAffine(degrees = 0, translate = (0.2, 0.2)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]), # normalization
    CLAHE(),
    GaussianNoise()

])
    
    train_dir = 'C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\cbis-train'
    train_dataset = datasets.ImageFolder(train_dir, transforms_train)
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)

    test_dir = 'C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\cbis-test'
    test_dataset = datasets.ImageFolder(test_dir, transforms_train)
    test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

    val_dir = 'C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\cbis-val'
    val_dataset = datasets.ImageFolder(val_dir, transforms_train)
    val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

    return train_dataloader,test_dataloader,val_dataloader


def build_resnet():
    model_ft = models.resnet50(weights='IMAGENET1K_V1')
    for param in model_ft.parameters():
        param.requires_grad = False
    num_ftrs = model_ft.fc.in_features
    print(num_ftrs)
    model_ft.fc = nn.Linear(num_ftrs, 1)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    return model_ft.to(device)

def build_densenet():
    model_ft = models.densenet169(weights='IMAGENET1K_V1')
    for param in model_ft.parameters():
        param.requires_grad = False
    num_ftrs = model_ft.fc.in_features

    model_ft.fc = nn.Linear(num_ftrs, 2)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    return model_ft.to(device)

        

def build_optimizer(network, optimizer, learning_rate):
    if optimizer == "sgd":
        optimizer = optim.SGD(network.parameters(),
                              lr=learning_rate, momentum=0.9)
    elif optimizer == "adam":
        optimizer = optim.Adam(network.parameters(),
                               lr=learning_rate)
    return optimizer


def train_epoch(network, loader, optimizer):
    cumu_loss = 0
    for _, (data, target) in enumerate(loader): # batch batch butun datayi gezip loss degerlerini hesapliyorum
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        # ➡ Forward pass
        loss = nn.CrossEntropyLoss()(network(data), target)
        cumu_loss += loss.item()

        # ⬅ Backward pass + weight update
        loss.backward()
        optimizer.step()


    return cumu_loss / len(loader)



def train(config=None):
    # Initialize a new wandb run
    with wandb.init(config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        train_dataloader,test_dataloader,val_dataloader = build_dataset(config.batch_size)
        network = build_resnet()
        optimizer = build_optimizer(network, config.optimizer, config.learning_rate)
        scheduler = lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.5, total_iters=10)
        es = EarlyStopping()
        best_val_loss = np.inf
        PATH = './cifar_net.pth'

        for epoch in range(config.epochs):
            network.train(True)
            avg_loss = train_epoch(network, train_dataloader, optimizer)
            scheduler.step()
            wandb.log({"training_loss": avg_loss, "epoch": epoch})    
            running_vloss = 0
            network.eval()
            loss_fn = nn.CrossEntropyLoss()
            with torch.no_grad():
                for i, vdata in enumerate(val_dataloader):
                    vinputs, vlabels = vdata
                    vinputs, vlabels = vinputs.to(device), vlabels.to(device)
                    voutputs = network(vinputs)
                    vloss = loss_fn(voutputs, vlabels)
                    running_vloss += vloss     

                avg_valloss = running_vloss / (i + 1) # avgvalloss stands for the validation loss value on i th epoch
                if es(network, avg_valloss):
                    done = True
                if avg_loss < best_val_loss:
                    best_val_loss = avg_loss
                    torch.save({
                                'epoch': epoch+1,
                                'model_state_dict': network.state_dict(),
                                'optimizer_state_dict': optimizer.state_dict()
                                
                }, PATH)
                wandb.log({"validation_loss":avg_valloss, "epoch": epoch})
        
        
        
        test()

resnet_predictions = []
def test():
    network = build_resnet()
    best_checkpoint = torch.load('./cifar_net.pth')
    best_epoch = best_checkpoint["epoch"]
    print("Best epoch",best_epoch)
    

    network.eval()
    y_true = []
    y_pred = []

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    train_dataloader,test_dataloader,val_dataloader = build_dataset(32)
    for epoch in range(best_epoch):
        with torch.no_grad():
            for images, labels in test_dataloader:
                images = images.to(device)
                outputs = network(images)
                _, predicted = torch.max(outputs, 1)
                resnet_predictions.extend(predicted.cpu().numpy())
                y_pred.extend(predicted.cpu().numpy()) # bu benim ust mlp ye koyacagim input oluyor su an
                y_true.extend(labels.cpu().numpy())
            

            # Convert lists to tensors for calculation
            y_true_tensor = torch.tensor(y_true)
            y_pred_tensor = torch.tensor(y_pred)
            
            # Calculating precision, recall, and F1 score using PyTorch
            #TP = ((y_pred_tensor == 1) & (y_true_tensor == 1)).sum().item()
            #FP = ((y_pred_tensor == 1) & (y_true_tensor == 0)).sum().item()
            #FN = ((y_pred_tensor == 0) & (y_true_tensor == 1)).sum().item()

            #precision = TP / (TP + FP) if TP + FP > 0 else 0
            #recall = TP / (TP + FN) if TP + FN > 0 else 0
            #f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
            #auc = np.round(roc_auc_score(y_true, y_pred), 3)

            #print(f'Precision: {precision}')
            #print(f'Recall: {recall}')
            #print(f'F1 Score: {f1}')
            
            #wandb.log({"precision":precision})
            #wandb.log({"recall":recall})
            #wandb.log({"f1 score":f1})
            #wandb.log({"AUC":auc})

test()
print(len(resnet_predictions))
print(resnet_predictions)



class EnsembeResult(nn.Module): # This neural network merges the 

    def __init__(self,model_resnet,model_catboost):
        super(EnsembeResult,self).__init__()
        self.model_resnet = model_resnet
        self.model_catboost = model_catboost
        self.classifier = nn.Linear(2,1)

    def forward(self,x1,x2):
        x1 = self.model_resnet(x1)
        x2 = self.model_catboost(x2)
        x = torch.cat((x1,x2),dim=1)
        x = self.classifier(F.relu(x))
        return x    
    

# Create models and load state_dicts    
#numericalClassifier = NumericalClassifier()
modelResnet50 = build_resnet()
# Load state dicts
PATH = './cifar_net.pth'
#numericalClassifier.load_state_dict(torch.load(PATH))
#modelResnet50.load_state_dict(torch.load(PATH))

#model = EnsembeResult(numericalClassifier, modelResnet50)



#output = model(x1, x2)


2048
Best epoch 2
464
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
train_dataloader,test_dataloader,val_dataloader = build_dataset(32)
for batch,val in test_dataloader:
    print(len(val))

In [20]:
sweep_config = {
    'method': 'grid'
    }

metric = {
    'name': 'loss',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

parameters_dict = {
    'optimizer': {
        'values': ['adam']
        },
    'learning_rate': {
        "values" : [0.001] # 0.0001 vermeyi bi dene bakalim
      },
      'batch_size': {
        "values" : [32]
      },
      "epochs" : {
        "values" : [1,2]
      }
    }
 
sweep_config['parameters'] = parameters_dict


import pprint
pprint.pprint(sweep_config)
sweep_id = wandb.sweep(sweep_config, project="Resnet50 Output")

#wandb.agent(sweep_id, train)



{'method': 'grid',
 'metric': {'goal': 'minimize', 'name': 'loss'},
 'parameters': {'batch_size': {'values': [32]},
                'epochs': {'values': [1, 2]},
                'learning_rate': {'values': [0.001]},
                'optimizer': {'values': ['adam']}}}
Create sweep with ID: o8erkbja
Sweep URL: https://wandb.ai/alphanhilal-university-of-sussex/Resnet50%20Output/sweeps/o8erkbja


In [35]:
numerical_predictions = []
def pathology_encoder(val):
    return "MALIGNANT" if val == "MALIGNANT" else "NORMAL"

calc_df = pd.read_csv("C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\csv\\calc_case_description_train_set.csv")
c_test = pd.read_csv("C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\csv\\calc_case_description_test_set.csv")
calc_df["pathology"] = calc_df["pathology"].map(pathology_encoder)
cols_to_be_used = ["breast density","left or right breast","calc type","calc distribution","pathology"]
new_calc_df = calc_df[cols_to_be_used]

mass_df = pd.read_csv("C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\csv\\mass_case_description_train_set.csv")
m_test = pd.read_csv("C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\csv\\mass_case_description_test_set.csv")
mass_df["pathology"] = mass_df["pathology"].map(pathology_encoder)
cols_to_be_used = ["breast_density","left or right breast","mass shape","mass margins","pathology"]
new_mass_df = mass_df[cols_to_be_used]
new_mass_df.rename(columns={"breast_density":"breast density"},inplace=True)

bc_df = pd.concat([new_calc_df,new_mass_df])
lbl_encoder = LabelEncoder()
bc_df['left or right breast'] = lbl_encoder.fit_transform(bc_df['left or right breast'])
bc_df['calc type'] = lbl_encoder.fit_transform(bc_df['calc type'])
bc_df["calc distribution"] = lbl_encoder.fit_transform(bc_df['left or right breast'])
bc_df['pathology'] = lbl_encoder.fit_transform(bc_df['pathology'])
bc_df['mass shape'] = lbl_encoder.fit_transform(bc_df['mass shape'])
bc_df['mass margins'] = lbl_encoder.fit_transform(bc_df['mass margins'])

bc_df["breast density"] = bc_df["breast density"].astype(float)

cols = ["breast density",'left or right breast', 'calc type','calc distribution', 'mass shape', 'mass margins']
scaler = StandardScaler()
bc_df[cols] = scaler.fit_transform(bc_df[cols])

x_train,y_train = bc_df["pathology"], bc_df["pathology"]
df = pd.concat([c_test,m_test])
df = df[["breast density","left or right breast","calc type","calc distribution","pathology","mass shape","mass margins"]] 
x_test,y_test = df.drop("pathology"), df["pathology"]

class NumericalClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(6, 32),
            nn.ReLU(),
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        output = self.linear_relu_stack(x)
        return output


y_train = np.reshape(y_train,(y_train.shape[0],1))
y_test = np.reshape(y_test,(y_test.shape[0],1))


train_set = np.concatenate((x_train,y_train),axis=1)
test_set = np.concatenate((x_test,y_test),axis=1)
train_set = torch.tensor(train_set, dtype=torch.float32)
test_set = torch.tensor(test_set, dtype=torch.float32)


batch_size = 32
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_set,  batch_size=batch_size, shuffle=False)
epochs = 10
optimizer = build_optimizer(NumericalClassifier(),"adam",learning_rate=0.001)
model = NumericalClassifier()
i = 0
for epoch in range(epochs):
    losses = []
    for batch_num, input_data in enumerate(train_loader):
        optimizer.zero_grad()
        x, y = input_data[:,:6], input_data[:,6]
        output = model(x)
        y = torch.reshape(y,(y.shape[0],1))
        loss = nn.BCELoss()(output, y)
        loss.backward()
        losses.append(loss.item())

        optimizer.step()

        if batch_num % 32 == 0:
            print('\tEpoch %d | Batch %d | Loss %6.2f' % (epoch, batch_num, loss.item()))
    print('Epoch %d | Loss %6.2f' % (epoch, sum(losses)/len(losses)))
    
    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for values in test_loader:
            labels = values[:,6] 
            outputs = model(values[:,:6])
            _, predicted = torch.max(outputs, 1)
            numerical_predictions.extend(predicted.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy()) # buradaki deger de ayni sekilde bir sonraki mlp ye koyacagim ikinci input oluyor.
            y_true.extend(labels.cpu().numpy())


        # Convert lists to tensors for calculation
        y_true_tensor = torch.tensor(y_true)
        y_pred_tensor = torch.tensor(y_pred)


        # Calculating precision, recall, and F1 score using PyTorch
        TP = ((y_pred_tensor == 1) & (y_true_tensor == 1)).sum().item()
        FP = ((y_pred_tensor == 1) & (y_true_tensor == 0)).sum().item()
        FN = ((y_pred_tensor == 0) & (y_true_tensor == 1)).sum().item()

        precision = TP / (TP + FP) if TP + FP > 0 else 0
        recall = TP / (TP + FN) if TP + FN > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        auc = np.round(roc_auc_score(y_true, y_pred), 3)

        print(f'Precision: {precision}')
        print(f'Recall: {recall}')
        print(f'F1 Score: {f1}')

print(numerical_predictions)



C:\Users\ae423\AppData\Local\Temp\ipykernel_23504\3278508086.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_mass_df.rename(columns={"breast_density":"breast density"},inplace=True)


	Epoch 0 | Batch 0 | Loss   0.71
	Epoch 0 | Batch 32 | Loss   0.71
	Epoch 0 | Batch 64 | Loss   0.71
Epoch 0 | Loss   0.71
Precision: 0
Recall: 0.0
F1 Score: 0
	Epoch 1 | Batch 0 | Loss   0.70
	Epoch 1 | Batch 32 | Loss   0.70
	Epoch 1 | Batch 64 | Loss   0.71
Epoch 1 | Loss   0.71
Precision: 0
Recall: 0.0
F1 Score: 0
	Epoch 2 | Batch 0 | Loss   0.72
	Epoch 2 | Batch 32 | Loss   0.73
	Epoch 2 | Batch 64 | Loss   0.70
Epoch 2 | Loss   0.71
Precision: 0
Recall: 0.0
F1 Score: 0
	Epoch 3 | Batch 0 | Loss   0.71
	Epoch 3 | Batch 32 | Loss   0.73
	Epoch 3 | Batch 64 | Loss   0.72
Epoch 3 | Loss   0.71
Precision: 0
Recall: 0.0
F1 Score: 0
	Epoch 4 | Batch 0 | Loss   0.70
	Epoch 4 | Batch 32 | Loss   0.69
	Epoch 4 | Batch 64 | Loss   0.71
Epoch 4 | Loss   0.71
Precision: 0
Recall: 0.0
F1 Score: 0
	Epoch 5 | Batch 0 | Loss   0.71
	Epoch 5 | Batch 32 | Loss   0.70
	Epoch 5 | Batch 64 | Loss   0.70
Epoch 5 | Loss   0.71
Precision: 0
Recall: 0.0
F1 Score: 0
	Epoch 6 | Batch 0 | Loss   0.70
	Epoch 

In [22]:
for i in range(len(numerical_predictions)):
    numerical_predictions[i] = numerical_predictions[i].item()
len(numerical_predictions)

5730

In [23]:
predictions = model.predict(x_test)
print(accuracy_score(predictions,y_test))
print(precision_score(predictions,y_test))
print(recall_score(predictions,y_test))
print(f1_score(predictions,y_test))

AttributeError: 'NumericalClassifier' object has no attribute 'predict'

In [55]:
bc_df

,breast density,left or right breast,calc type,calc distribution,pathology,mass shape,mass margins
0,3,RIGHT,AMORPHOUS,CLUSTERED,MALIGNANT,NaN,NaN
1,3,RIGHT,AMORPHOUS,CLUSTERED,MALIGNANT,NaN,NaN
2,4,LEFT,PLEOMORPHIC,LINEAR,NORMAL,NaN,NaN
3,4,LEFT,PLEOMORPHIC,LINEAR,NORMAL,NaN,NaN
4,1,LEFT,NaN,REGIONAL,NORMAL,NaN,NaN
...,...,...,...,...,...,...,...
1313,2,RIGHT,NaN,NaN,MALIGNANT,IRREGULAR,ILL_DEFINED
1314,2,RIGHT,NaN,NaN,MALIGNANT,ROUND,SPICULATED
1315,2,RIGHT,NaN,NaN,MALIGNANT,ROUND,SPICULATED
1316,2,LEFT,NaN,NaN,MALIGNANT,IRREGULAR,SPICULATED
